## Web Scrapping Dog Database

The objective of this 

### Libraries

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import io
from PIL import Image
import time

dogs_ig = [
    '@itsdougthepug',
    '@jiffpom',
    '@marniethedog',
    '@manny_the_frenchie',
    '@crusoe_dachshund',
    '@samsonthedood',
    '@reagandoodle',
    '@barkleysircharles',
    '@popeyethefoodie',
    '@izzythe.frenchie',
    '@tunameltsmyheart',
    '@toastmeetsworld',
    '@mensweardog',
    '@thiswildidea',
    '@aspenthemountainpup',
    '@dogwithsign',
    '@goldenunicornrae',
    '@jackson_the_dalmatian',
    '@madmax_fluffyroad',
    '@pavlovthecorgi',
    '@tuckerbudzyn',
    '@ppteamkler',
    '@Theladyshortcake',
    '@rocco_roni',
    '@Chompersthecorgi',
    '@siberianhusky_jax',
    '@good.boy.ollie',
    '@bluestaffyboulder',
    '@lecorgi',
    '@carterchowchow',
    '@_gsdbear',
    '@harlso',
    '@KeyushTheStuntDog',
    '@mayapolarbear',
    '@marutaro',
    '@henrythecoloradodog',
    '@eddie_jackrussell',
    '@tecuaniventura',
    '@ppteamaria',
    '@emmatheminifrenchie',
    '@balooitsme',
    '@pipperontour',
    '@mayathedox',
    '@hi_im_chewie',
    '@frankietothemoon',
    '@tinkerbellethedog',
    '@loki_the_wolfdog',
    '@dailydougie',
    '@tikatheiggy',
    '@norbertthedog'
]

def get_duplicates(lst):
    return list(set([x for x in lst if lst.count(x) > 1]))

print(get_duplicates(dogs_ig))
print(len(dogs_ig))


[]
50


In [3]:

def get_images_from_google(wd, query, delay, max_images):

	def scroll_down(wd):
		wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
		time.sleep(delay)

	query = query.replace('@', '')
	
	url = f"https://www.google.com/search?q={query}&tbm=isch"
	wd.get(url)

	image_urls = set()
	skips = 0

	while len(image_urls) + skips < max_images:
		scroll_down(wd)

		thumbnails = wd.find_elements(By.CLASS_NAME, "Q4LuWd")

		for img in thumbnails[len(image_urls) + skips:max_images]:
			try:
				img.click()
				time.sleep(delay)
			except:
				continue

			images = wd.find_elements(By.CLASS_NAME, "sFlh5c")
			for image in images:
				if image.get_attribute('src') in image_urls:
					max_images += 1
					skips += 1
					break

				if image.get_attribute('src') and 'http' in image.get_attribute('src'):
					image_urls.add(image.get_attribute('src'))
					print(f"Found {len(image_urls)}")

	return image_urls


def download_image(download_path, url, file_name):
	try:
		image_content = requests.get(url).content
		image_file = io.BytesIO(image_content)
		image = Image.open(image_file)
		file_path = download_path + file_name

		with open(file_path, "wb") as f:
			image.save(f, "JPEG")

		print("Success")
	except Exception as e:
		print('FAILED -', e)




def download_dogs_images(wd, dogs_ig, delay, max_images):
	for dog in dogs_ig:
		query = dog
		urls = get_images_from_google(wd, query, delay, max_images)

		for i, url in enumerate(urls):
			download_image("imgs/", url, str(dog) + str(i) + ".jpg")

In [4]:
print(dogs_ig[45:])

['@tinkerbellethedog', '@loki_the_wolfdog', '@dailydougie', '@tikatheiggy', '@norbertthedog']


In [5]:
import os
from PIL import Image
import pandas as pd

# Path to the folder containing images
folder_path = "imgs/"

# List all files in the folder
files = os.listdir(folder_path)

# Initialize lists to store file names and dimensions
file_names = []
dimensions = []
file_sizes = []
aspect_ratios = []

# Iterate through each file
for file in files:
    # Check if the file is an image
    if file.endswith((".png", ".jpg", ".jpeg", ".gif")):
        # Get the full path of the image
        image_path = os.path.join(folder_path, file)
        
        # Open the image
        with Image.open(image_path) as img:
            # Get the dimensions
            width, height = img.size
            # Calculate aspect ratio
            aspect_ratio = width / height
            # Get file size
            file_size = os.path.getsize(image_path) / (1024 * 1024)  # Convert to MB
            # Append file name, dimensions, file size, and aspect ratio to the lists
            file_names.append(file)
            dimensions.append((width, height))
            file_sizes.append(file_size)
            aspect_ratios.append(aspect_ratio)

# Create a DataFrame
df = pd.DataFrame({"File Name": file_names, 
                   "Dimensions": dimensions, 
                   "File Size (MB)": file_sizes,
                   "Aspect Ratio": aspect_ratios})
df = df.sort_values(by="File Name")

# Export the DataFrame to a CSV file
df.to_csv("image_data.csv", index=False)